# P-values

This is the part in which we run the p-value calculation.

The p-value calculation is very simple (see https://arxiv.org/pdf/1007.1727.pdf).

It's given by:
$q_0 = 2($NLL$_0 - $NLL$)$

where NLL$_0$ is the negative log-likelihood calculated when the POI is set to zero (background-only hypothesis), and NLL is the value when also the POI is free to float.

Let's first load the workspace, the model-config and the dataset.

In [ ]:
f = new TFile("../ws/ICTPws_test_combined_ICTPws_test_model.root");
w = dynamic_cast<RooWorkspace*>(f->Get("combined"));
mc = dynamic_cast<RooStats::ModelConfig*>(w->obj("ModelConfig"));
dataset = w->data("obsData");
w->pdf("simPdf")->fitTo(*dataset)

Then perform the *conditional fit*: 

we fix the POI to 0 and we minimize the NLL, i.e. we run the fit in the background-only hypothesis. 

NB: we need to also save the fit results.

In [ ]:
w->var("mu_ttH")->setVal(0);
w->var("mu_ttH")->setConstant(kTRUE);
RooFitResult *r = w->pdf("simPdf")->fitTo(*dataset, RooFit::Save());

From the `FitResult` instance that we saved, we extract the negative-log-likelihood (NLL) value:

In [ ]:
nll_0 = r->minNll();
cout << nll_0 << endl;

Now do an unconditional fit:

In [ ]:
w->var("mu_ttH")->setVal(1);
w->var("mu_ttH")->setConstant(kFALSE);
RooFitResult *r_mu = w->pdf("simPdf")->fitTo(*dataset, RooFit::Save());

In [ ]:
nll_mu = r_mu->minNll();
cout << nll_mu << endl;

At this point, we get the difference between the two NLL values:

In [ ]:
cout << "delta NLL = " << nll_0 - nll_mu << endl;

and we tell ROOT to calculate the p-value and the significance from it:

In [ ]:
float p0 = TMath::Prob(2*(nll_0 - nll_mu),1);
float Z = RooStats::PValueToSignificance(p0);
cout << "p-value      = " << p0 << endl;
cout << "Significance = " << Z << endl;